### Scraping lyric data from metal-archives.com

In [ ]:
import requests
import lxml.html as lh
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd

In [68]:
def bands_on_page(url):
    headers = { 'User-Agent': 'Mozilla/5.0'
    }
    response = requests.get(url, headers=headers)
    bs = BeautifulSoup(response.text,'lxml')
    band_links = []
    for link in bs.find_all('a'):
        band_url = str(link).split('\"')[1]
        band_id = band_url.split('/')[-1][:-1]
        band_disco = f'https://www.metal-archives.com/band/discography/id/{band_id}/tab/all'
        band_links.append(band_disco)
    return band_links

band_links = []
for i in range(0, 37800, 200):
    URL = f'https://www.metal-archives.com/search/ajax-advanced/searching/bands/?bandName=&genre=&country=US&yearCreationFrom=&yearCreationTo=&bandNotes=&status=&themes=&location=&bandLabelName=&sEcho=4&iColumns=3&sColumns=&iDisplayStart={i}&iDisplayLength=37800&mDataProp_0=0&mDataProp_1=1&mDataProp_2=2&_=1692040038976'
    band_links = np.concatenate((band_links, bands_on_page(URL)))

In [69]:
print(len(band_links))


37755


In [ ]:

https://www.metal-archives.com/search/ajax-advanced/searching/bands/?bandName=&genre=&country=US&yearCreationFrom=&yearCreationTo=&bandNotes=&status=&themes=&location=&bandLabelName=&sEcho=2&iColumns=3&sColumns=&iDisplayStart=200&iDisplayLength=200&mDataProp_0=0&mDataProp_1=1&mDataProp_2=2&_=1692040038974

https://www.metal-archives.com/search/ajax-advanced/searching/bands/?bandName=&genre=&country=US&yearCreationFrom=&yearCreationTo=&bandNotes=&status=&themes=&location=&bandLabelName=&sEcho=1&iColumns=3&sColumns=&iDisplayStart=0&iDisplayLength=200&mDataProp_0=0&mDataProp_1=1&mDataProp_2=2&_=1692040038973


https://www.metal-archives.com/search/ajax-advanced/searching/bands/?bandName=&genre=&country=US&yearCreationFrom=&yearCreationTo=&bandNotes=&status=&themes=&location=&bandLabelName=&sEcho=3&iColumns=3&sColumns=&iDisplayStart=400&iDisplayLength=200&mDataProp_0=0&mDataProp_1=1&mDataProp_2=2&_=1692040038975